In [47]:
from sklearn.model_selection import RandomizedSearchCV

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical 


from sklearn import ensemble

import os

# HBP
data = pd.read_csv('hbp.txt', sep=">",header=None)
sequences=data[0].dropna()
labels=data[1].dropna()
sequences.reset_index(drop=True, inplace=True)
labels.reset_index(drop=True, inplace=True)
list_of_series=[sequences.rename("sequences"),labels.rename("Name")]
df_hbp = pd.concat(list_of_series, axis=1)
df_hbp['label']='hbp'
df_hbp.head()

# not HBP
data = pd.read_csv('non-hbp.txt', sep=">",header=None)
sequences=data[0].dropna()
labels=data[1].dropna()
sequences.reset_index(drop=True, inplace=True)
labels.reset_index(drop=True, inplace=True)
list_of_series=[sequences.rename("sequences"),labels.rename("Name")]
df_N_hbp = pd.concat(list_of_series, axis=1)
df_N_hbp['label']='non-hbp'
df_N_hbp.head()


frames = [df_hbp,df_N_hbp]
df=pd.concat(frames)
df.head()

from sklearn.preprocessing import LabelBinarizer
import keras

lb = LabelBinarizer()
Y = lb.fit_transform(df.label)



def frequencyVec(seq):
    encoder = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W',
               'Y']
    fv = [0 for x in range(20)]
    i = 0
    for i in range(20):
        fv[i - 1] = seq.count(encoder[i])
    return fv
X_frequencyVec=[]
for i in df.sequences:
    X_frequencyVec.append(frequencyVec(i))
                      
                      
X_frequencyVec = np.asarray(X_frequencyVec)
X_frequencyVec.shape
                    
X_train, X_test, y_train, y_test = train_test_split(X_frequencyVec, df.label, test_size=.3, random_state=0)
clfBC=ensemble.BaggingClassifier()

base_estimator=None
n_estimators=30
warm_start=True
bootstrap=True
bstlfy=ensemble.BaggingClassifier(base_estimator=base_estimator,n_estimators=n_estimators,warm_start=warm_start,bootstrap=bootstrap)
bstlfy=bstlfy.fit(X_train, y_train)
prediction=bstlfy.predict(X_test)



In [48]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(bstlfy, open(filename, 'wb'))
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.7297297297297297
